In [6]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.2.0


# Parameter setting

In [25]:
IMAGE_SIZE = [448, 448]
EMBEDDING_SIZE = 512

# TPU

In [16]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


# Dataset 관련

In [9]:
def get_paths(sub):
    index = ["0","1","2","3","4","5","6","7","8","9","a","b","c","d","e","f"]

    paths = []

    for a in index:
        for b in index:
            for c in index:
                try:
                    paths.extend([f"../input/landmark-retrieval-2020/{sub}/{a}/{b}/{c}/" + x for x in os.listdir(f"../input/landmark-retrieval-2020/{sub}/{a}/{b}/{c}")])
                except:
                    pass

    return paths

train_set_paths = get_paths('train')
print('The number of train images is {}'.format(len(train_set_paths)))

The number of train images is 3147


In [10]:
train_df = pd.read_csv("../input/landmark-retrieval-2020/train.csv")
print(len(train_df))
train_df = train_df.sort_values(by=['landmark_id'])

label_set = {}

cnt = 0
for label in train_df['landmark_id']:
    if label not in label_set:
        label_set[label] = cnt
        cnt += 1

print('The number of unique labels is {}'.format(len(label_set)))

3147
The number of unique labels is 2916


# Model setting

In [45]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

from tensorflow.keras.layers import Layer, Dense, BatchNormalization
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
import tensorflow_probability as tfp

class AdaCos(Layer):
    def __init__(self, n_classes=10, regularizer=None, **kwargs):
        super(AdaCos, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = math.sqrt(2)*math.log(n_classes-1)
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        print(input_shape)
        print(input_shape)
        super(AdaCos, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        print(x)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        print(y)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))

        B_avg = tf.where(y < 1, tf.exp(self.s*logits), tf.zeros_like(logits))
        B_avg = tf.reduce_mean(tf.reduce_sum(B_avg, axis=1), name='B_avg')
        theta_class = tf.gather(theta, tf.cast(y, tf.int32), name='theta_class')
        theta_med = tfp.stats.percentile(theta_class, q=50)

        with tf.control_dependencies([theta_med, B_avg]):
            self.s = tf.math.log(B_avg) / tf.cos(tf.minimum(math.pi/4, theta_med))
            logits = self.s * logits 
            out = tf.nn.softmax(logits)
        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

class Generalized_mean_pooling2D(Layer):
    def __init__(self, p=3, epsilon=1e-6, **kwargs):
        super(Generalized_mean_pooling2D, self).__init__(**kwargs)

        self.init_p = p
        self.epsilon = epsilon

    def build(self, input_shape):
        if isinstance(input_shape, list) or len(input_shape) != 4:
            raise ValueError('`GeM` pooling layer only allow 1 input with 4 dimensions(b, h, w, c)')

        self.build_shape = input_shape

        self.p = self.add_weight(
              name='p',
              shape=[1,],
              initializer=tf.keras.initializers.Constant(value=self.init_p),
              regularizer=None,
              trainable=True,
              dtype=tf.float32
              )

        self.built=True

    def call(self, inputs):
        input_shape = inputs.get_shape()
        if isinstance(inputs, list) or len(input_shape) != 4:
            raise ValueError('`GeM` pooling layer only allow 1 input with 4 dimensions(b, h, w, c)')

        return (tf.reduce_mean(tf.abs(inputs**self.p), axis=[1,2], keepdims=False) + self.epsilon)**(1.0/self.p)

with strategy.scope():
    enet = efn.EfficientNetB7(
        weights='noisy-student',
        include_top=False
    )
    enet.trainable = True

    embed_model = tf.keras.Sequential([
        enet,
        Generalized_mean_pooling2D(name='GeM'),
        Dense(EMBEDDING_SIZE, activation='softmax', name='fc'),
        BatchNormalization(name='batchnorm')
    ])

embed_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy',  
              metrics=['sparse_categorical_accuracy']
)

embed_model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, None, None, 2560)  64097680  
_________________________________________________________________
GeM (Generalized_mean_poolin (None, 2560)              1         
_________________________________________________________________
fc (Dense)                   (None, 512)               1311232   
_________________________________________________________________
batchnorm (BatchNormalizatio (None, 512)               2048      
Total params: 65,410,961
Trainable params: 65,099,217
Non-trainable params: 311,744
_________________________________________________________________
